In [1]:
!pip install datasets
!pip install accelerate -U
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

# Building Question Answering Datasets
In this exercise, we want to construct a dataset for training a question answering model.
In general, this process is highly manual, requiring the collection of data and, as one might expect, manually constructing pairs of questions and answers that can be found in the text.
In our exercise, we provide the data plus question and answer pairs, and work through how to construct the dataset from that starting point.

## SQuAD
One of the most well studied datasets in question answering is the Stanford Question Answering Dataset (SQuAD), introduced in a paper by [Rajpurkar et al.](https://nlp.stanford.edu/pubs/rajpurkar2016squad.pdf) and its follow-up, [SQuAD 2.0](https://arxiv.org/abs/1806.03822).
Formatting our dataset like SQuAD makes it much easier to use prebuild models and trainers like those from [HuggingFace](https://huggingface.co/) to do our job, so it makes sense to first understand the structure of the dataset.
What follows is a single example from the dataset.

``` python
{'id': '573387acd058e614000b5cb5',
 'title': 'University_of_Notre_Dame',
 'context': 'One of the main driving forces in the growth of the University was its football team, the Notre Dame Fighting Irish. Knute Rockne became head coach in 1918. Under Rockne, the Irish would post a record of 105 wins, 12 losses, and five ties. During his 13 years the Irish won three national championships, had five undefeated seasons, won the Rose Bowl in 1925, and produced players such as George Gipp and the "Four Horsemen". Knute Rockne has the highest winning percentage (.881) in NCAA Division I/FBS football history. Rockne\'s offenses employed the Notre Dame Box and his defenses ran a 7–2–2 scheme. The last game Rockne coached was on December 14, 1930 when he led a group of Notre Dame all-stars against the New York Giants in New York City.',
 'question': 'In what year did the team lead by Knute Rockne win the Rose Bowl?',
 'answers': {'text': ['1925'],
             'answer_start': [354]}
}
 ```

 As we can see, each entry in the dataset is a Python `dict` object with the structure:

 ``` python
 {'id': str,
  'title': str,
  'context': str,
  'question': str,
  'answers': dict
 }
 ```

For our purposes, we can simply ignore the 'title' entry if we wish -- it has no impact on our ability to use the dataset, but can be important for others, as it maps to the title of the article from which the context is drawn.
The answers are themselves a dict with structure:

``` python
{'text': list(str),
 'answer_start': list(int)
}
```
This is important since there may be multiple answers in a single context.

## Imports
Let's start by importing the libraries we're going to need.

# Construindo conjuntos de dados de respostas a perguntas
Neste exercício, queremos construir um conjunto de dados para treinar um modelo de resposta a perguntas.
Em geral, esse processo é altamente manual, exigindo a coleta de dados e, como seria de esperar, a construção manual de pares de perguntas e respostas que podem ser encontradas no texto.
Em nosso exercício, fornecemos os dados mais os pares de perguntas e respostas e estudamos como construir o conjunto de dados a partir desse ponto de partida.

## Esquadrão
Um dos conjuntos de dados mais bem estudados em resposta a perguntas é o Stanford Question Answering Dataset (SQuAD), apresentado em um artigo de [Rajpurkar et al.](https://nlp.stanford.edu/pubs/rajpurkar2016squad.pdf) e seu acompanhamento, [SQuAD 2.0](https://arxiv.org/abs/1806.03822).
A formatação do nosso conjunto de dados como SQuAD torna muito mais fácil usar modelos pré-construídos e treinadores como os do [HuggingFace](https://huggingface.co/) para fazer nosso trabalho, por isso faz sentido primeiro entender a estrutura do conjunto de dados.
O que se segue é um único exemplo do conjunto de dados.

``` píton
{'id': '573387acd058e614000b5cb5',
  'título': 'Universidade_de_Notre_Dame',
  'contexto': 'Uma das principais forças motrizes no crescimento da Universidade foi o seu time de futebol, o Notre Dame Fighting Irish. Knute Rockne tornou-se treinador principal em 1918. Sob Rockne, os irlandeses registrariam um recorde de 105 vitórias, 12 derrotas e cinco empates. Durante seus 13 anos, o irlandês venceu três campeonatos nacionais, teve cinco temporadas invicto, venceu o Rose Bowl em 1925 e produziu jogadores como George Gipp e os "Quatro Cavaleiros". Knute Rockne tem a maior porcentagem de vitórias (0,881) na história do futebol da Divisão I/FBS da NCAA. Os ataques de Rockne empregaram o Notre Dame Box e suas defesas seguiram um esquema de 7–2–2. O último jogo que Rockne treinou foi em 14 de dezembro de 1930, quando liderou um grupo de estrelas de Notre Dame contra o New York Giants na cidade de Nova York.',
  'pergunta': 'Em que ano a equipe liderada por Knute Rockne ganhou o Rose Bowl?',
  'respostas': {'texto': ['1925'],
              'resposta_início': [354]}
}
  ```

  Como podemos ver, cada entrada no conjunto de dados é um objeto `dict` do Python com a estrutura:

  ``` píton
  {'id':str,
   'título': str,
   'contexto': str,
   'pergunta': str,
   'respostas': ditado
  }
  ```

Para os nossos propósitos, podemos simplesmente ignorar a entrada 'título' se desejarmos - ela não tem impacto na nossa capacidade de usar o conjunto de dados, mas pode ser importante para outros, pois mapeia para o título do artigo do qual o contexto é desenhado.
As próprias respostas são um ditado com estrutura:

``` píton
{'texto': lista(str),
  'resposta_início': lista(int)
}
```
Isto é importante porque pode haver múltiplas respostas em um único contexto.

## Importações
Vamos começar importando as bibliotecas que precisaremos.
Enviar feedback
Painéis laterais
Histórico
Salvas
Contribuir

In [2]:
from pathlib import Path
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, default_data_collator, pipeline
from transformers.trainer_utils import PredictionOutput
import math
import time
import collections
import numpy as np
from tqdm.notebook import tqdm

## Building our Dictionaries
Our question-answer pairs are stored in a CSV with columns: "question", "answer" and "filename".

## Construindo nossos dicionários
Nossos pares pergunta-resposta são armazenados em um CSV com colunas: "pergunta", "resposta" e "nome do arquivo".

In [3]:
df = pd.read_csv("data/qa.csv")
df.head()

,question,answer,filename
0,Who is the manufacturer of the product?,Zyxel,CVE-2020-29583.txt
1,Who reported the vulnerability?,researchers from EYE Netherlands,CVE-2020-29583.txt
2,What is the vulnerability?,A hardcoded credential vulnerability was ident...,CVE-2020-29583.txt
3,How do users protect themselves?,we urge users to install the applicable updates,CVE-2020-29583.txt
4,What products are affected?,firewalls and AP controllers,CVE-2020-29583.txt


Given the source of our question-answer pairs, we'll want a function that takes the question, answer, and filepath, and returns a nicely formatted dictionary.

Since we want each question to have a unique identifier, we will also take that as a function argument so we can use an iterator.

Within the function, we also want to read the context from the provided file and locate the starting index of the provided answer.
We can do this with the `.find()` method of the context string object.

If you wish to specify a 'title' in your object, I recommend using the filepath of the context, e.g. "data/filename" because the files are stored in a "data" folder.

Dada a origem dos nossos pares pergunta-resposta, precisaremos de uma função que pegue a pergunta, a resposta e o caminho do arquivo e retorne um dicionário bem formatado.

Como queremos que cada pergunta tenha um identificador único, também consideraremos isso como um argumento de função para que possamos usar um iterador.

Dentro da função, também queremos ler o contexto do arquivo fornecido e localizar o índice inicial da resposta fornecida.
Podemos fazer isso com o método `.find()` do objeto string de contexto.

Se você deseja especificar um 'título' no seu objeto, recomendo usar o caminho do arquivo do contexto, por exemplo. "data/filename" porque os arquivos são armazenados em uma pasta "data".

In [5]:
def qa_to_squad(question, answer, filename, identifier):
    filepath = "data/" + row['filename']
    with open(filepath, "r") as f:
        context = f.read()

    start_location = context.find(answer)
    qa_pair = {
        'id': identifier,
        'title': filepath,
        'context': context,
        'question': question,
        'answers': {'text': [answer],
                   'answer_start': [start_location]}
    }
    return qa_pair

## Building Dictionaries from Data
Now that we can turn our CSV of questions, answers, and filenames into dictionaries, we'll need to construct a dictionary for each question and answer pair in our file.

Let's use the `iterrows()` method of the dataframe to iterate through each question and answer pair, and use the `qa_to_squad` function to generate a SQuAD-formatted dictionary. Then we will append the formatted dictionary onto a list. At the end, you will have a list of SQuAD-formatted questions and answers.

In production, you may want to use the `uuid` library, or take the hash of the question + context + answers to ensure unique identifiers can be reconstructed. To simplify the process, we can use the index of the row from the `iterrows()` method.

## Construindo Dicionários a partir de Dados
Agora que podemos transformar nosso CSV de perguntas, respostas e nomes de arquivos em dicionários, precisaremos construir um dicionário para cada par de perguntas e respostas em nosso arquivo.

Vamos usar o método `iterrows()` do dataframe para iterar cada par de perguntas e respostas e usar a função `qa_to_squad` para gerar um dicionário formatado em SQuAD. Em seguida, anexaremos o dicionário formatado a uma lista. Ao final, você terá uma lista de perguntas e respostas formatadas no SQuAD.

Na produção, você pode usar a biblioteca `uuid` ou usar o hash da pergunta + contexto + respostas para garantir que identificadores exclusivos possam ser reconstruídos. Para simplificar o processo, podemos usar o índice da linha do método `iterrows()`.

In [14]:
qa_list = list()

# Todo: iterate over the rows of your dataframe to construct a SQuAD-formatted dictionary
for i, row in df.iterrows():
    question = row['question']
    answer = row['answer']
    filename = row['filename']

    # Todo: use qa_to_squad function to generate a SQuAD-formatted dictionary
    SQuAD_dict = qa_to_squad(question, answer, filename, i)

    # Append the dictionary to the qa_list
    qa_list.append(SQuAD_dict)

## Dicts to Datasets
Unfortunately, HuggingFace's `datasets` library does not have a great way to turn a list of dictionaries into a Dataset.
Luckily for us, `datasets` plays very nicely with `pandas`, which is happy to construct a DataFrame from a list of dictionaries.
First, we'll need to turn our list of SQuAD-formatted questions into a DataFrame.
Then, we'll need to use the `Dataset` `.from_pandas` method to create our HuggingFace-friendly dataset!

## Ditos para conjuntos de dados
Infelizmente, a biblioteca `datasets` do HuggingFace não tem uma ótima maneira de transformar uma lista de dicionários em um conjunto de dados.
Felizmente para nós, `datasets` funciona muito bem com `pandas`, que fica feliz em construir um DataFrame a partir de uma lista de dicionários.
Primeiro, precisaremos transformar nossa lista de perguntas formatadas no SQuAD em um DataFrame.
Então, precisaremos usar o método `Dataset` `.from_pandas` para criar nosso conjunto de dados compatível com HuggingFace!

In [16]:
# Create a pandas DataFrame object from your list of questions and answers
qa_df = pd.DataFrame(data=qa_list)

# Create a HuggingFace Dataset from that DataFrame
data = Dataset.from_pandas(qa_df)

In [17]:
# Print an object from your dataset by specifying an identifier
print(data[0])

{'id': 0, 'title': 'data/CVE-2020-29583.txt', 'context': 'CVE:   CVE-2020-29583 Summary Zyxel has released a patch for the hardcoded credential vulnerability of firewalls and AP controllers recently reported by researchers from EYE Netherlands. Users are advised to install the applicable firmware updates for optimal protection. What is the vulnerability? A hardcoded credential vulnerability was identified in the “zyfwp” user account in some Zyxel firewalls and AP controllers. The account was designed to deliver automatic firmware updates to connected access points through FTP. What versions are vulnerable—and what should you do? After a thorough investigation, we’ve identified the vulnerable products and are releasing firmware patches to address the issue, as shown in the table below. For optimal protection, we urge users to install the applicable updates. For those not listed, they are not affected. Contact your local Zyxel support team if you require further assistance or visit our  

## Saving our Dataset
Having done all this hard work, the last thing we want to do is have to reconstruct our dataset from scratch at run time.
To do this, call the `.save_to_disk()` method on your `Dataset` object

## Salvando nosso conjunto de dados
Depois de fazer todo esse trabalho árduo, a última coisa que queremos é reconstruir nosso conjunto de dados do zero em tempo de execução.
Para fazer isso, chame o método `.save_to_disk()` em seu objeto `Dataset`

In [18]:
data.save_to_disk("qa_data.hf")

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

## Fine Tuning BERT on our Data
Now that we have our dataset, let's fine-tune a pretrained model on it! This code is written for you, but we're using HuggingFace's `transformers` library and the `AutoModelForQuestionAnswering` to do the training here.

**NOTE:** This section is a simple example showing you how the data can be used to fine-tune a pretrained model. You don't need to modify any of the code below.

## Ajuste fino do BERT em nossos dados
Agora que temos nosso conjunto de dados, vamos ajustar um modelo pré-treinado nele! Este código foi escrito para você, mas estamos usando a biblioteca `transformers` do HuggingFace e o `AutoModelForQuestionAnswering` para fazer o treinamento aqui.

**NOTA:** Esta seção é um exemplo simples que mostra como os dados podem ser usados para ajustar um modelo pré-treinado. Você não precisa modificar nenhum código abaixo.

In [19]:
# Load the tokenizer for DistilBERT
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Load the model
# Note: This will throw warnings, which is expected!
model = AutoModelForQuestionAnswering.from_pretrained('distilbert-base-uncased')

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# The Trainer subclass here is lightly modified from HuggingFace
# Original source at https://github.com/huggingface/transformers/blob/main/examples/pytorch/question-answering/trainer_qa.py
class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, post_process_function=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.post_process_function = post_process_function

    def predict(self, predict_dataset, predict_examples, ignore_keys=None, metric_key_prefix: str = "test"):
        predict_dataloader = self.get_test_dataloader(predict_dataset)

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                predict_dataloader,
                description="Prediction",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )

        if self.post_process_function is None or self.compute_metrics is None:
            return output

        predictions = self.post_process_function(predict_examples, predict_dataset, output.predictions, "predict")
        metrics = self.compute_metrics(predictions)

        # Prefix all keys with metric_key_prefix + '_'
        for key in list(metrics.keys()):
            if not key.startswith(f"{metric_key_prefix}_"):
                metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
        metrics.update(output.metrics)
        return PredictionOutput(predictions=predictions.predictions, label_ids=predictions.label_ids, metrics=metrics)

## Pre- and Post-processing Functions
The following functions are lightly adapted from the HuggingFace `run_qa.py` example [on Github](https://github.com/huggingface/transformers/blob/main/examples/pytorch/question-answering/run_qa.py)

## Funções de pré e pós-processamento
As funções a seguir foram levemente adaptadas do exemplo HuggingFace `run_qa.py` [no Github](https://github.com/huggingface/transformers/blob/main/examples/pytorch/question-answering/run_qa.py)

In [24]:
# Training preprocessing
def prepare_train_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=512,
        padding="max_length",
        return_offsets_mapping=True
    )

    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        answers = examples["answers"][i]

        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1

            # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
            # Note: we could go after the last offset if the answer is the last word (edge case).
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def postprocess_qa_predictions(
    examples,
    features,
    predictions,
    version_2_with_negative = False,
    n_best_size = 20,
    max_answer_length = 30,
    null_score_diff_threshold = 0.0,
):
    """
    Post-processes the predictions of a question-answering model to convert them to answers that are substrings of the
    original contexts. This is the base postprocessing functions for models that only return start and end logits.

    Args:
        examples: The non-preprocessed dataset (see the main script for more information).
        features: The processed dataset (see the main script for more information).
        predictions (:obj:`Tuple[np.ndarray, np.ndarray]`):
            The predictions of the model: two arrays containing the start logits and the end logits respectively. Its
            first dimension must match the number of elements of :obj:`features`.
        version_2_with_negative (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether or not the underlying dataset contains examples with no answers.
        n_best_size (:obj:`int`, `optional`, defaults to 20):
            The total number of n-best predictions to generate when looking for an answer.
        max_answer_length (:obj:`int`, `optional`, defaults to 30):
            The maximum length of an answer that can be generated. This is needed because the start and end predictions
            are not conditioned on one another.
        null_score_diff_threshold (:obj:`float`, `optional`, defaults to 0):
            The threshold used to select the null answer: if the best answer has a score that is less than the score of
            the null answer minus this threshold, the null answer is selected for this example (note that the score of
            the null answer for an example giving several features is the minimum of the scores for the null answer on
            each feature: all features must be aligned on the fact they `want` to predict a null answer).

            Only useful when :obj:`version_2_with_negative` is :obj:`True`.
    """
    if len(predictions) != 2:
        raise ValueError("`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    if version_2_with_negative:
        scores_diff_json = collections.OrderedDict()

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]
            # Optional `token_is_max_context`, if provided we will remove answers that do not have the maximum context
            # available in the current feature.
            token_is_max_context = features[feature_index].get("token_is_max_context", None)

            # Update minimum null prediction.
            feature_null_score = start_logits[0] + end_logits[0]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[0],
                    "end_logit": end_logits[0],
                }

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or len(offset_mapping[start_index]) < 2
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[end_index]) < 2
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    # Don't consider answer that don't have the maximum context available (if such information is
                    # provided).
                    if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )
        if version_2_with_negative and min_null_prediction is not None:
            # Add the minimum null prediction
            prelim_predictions.append(min_null_prediction)
            null_score = min_null_prediction["score"]

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

        # Add back the minimum null prediction if it was removed because of its low score.
        if (
            version_2_with_negative
            and min_null_prediction is not None
            and not any(p["offsets"] == (0, 0) for p in predictions)
        ):
            predictions.append(min_null_prediction)

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0] : offsets[1]]

        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
        # failure.
        if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

        # Compute the softmax of all scores (we do it with numpy to stay independent from torch/tf in this file, using
        # the LogSumExp trick).
        scores = np.array([pred.pop("score") for pred in predictions])
        exp_scores = np.exp(scores - np.max(scores))
        probs = exp_scores / exp_scores.sum()

        # Include the probabilities in our predictions.
        for prob, pred in zip(probs, predictions):
            pred["probability"] = prob

        # Pick the best prediction. If the null answer is not possible, this is easy.
        if not version_2_with_negative:
            all_predictions[example["id"]] = predictions[0]["text"]
        else:
            # Otherwise we first need to find the best non-empty prediction.
            i = 0
            while predictions[i]["text"] == "":
                i += 1
            best_non_null_pred = predictions[i]

            # Then we compare to the null prediction using the threshold.
            score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
            scores_diff_json[example["id"]] = float(score_diff)  # To be JSON-serializable.
            if score_diff > null_score_diff_threshold:
                all_predictions[example["id"]] = ""
            else:
                all_predictions[example["id"]] = best_non_null_pred["text"]

        # Make `predictions` JSON-serializable by casting np.float back to float.
        all_nbest_json[example["id"]] = [
            {k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items()}
            for pred in predictions
        ]

    return all_predictions

# Post-processing:
def post_processing_function(examples, features, predictions, stage="eval"):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=data_args.version_2_with_negative,
        n_best_size=data_args.n_best_size,
        max_answer_length=data_args.max_answer_length,
        null_score_diff_threshold=data_args.null_score_diff_threshold,
        output_dir=training_args.output_dir,
        log_level=log_level,
        prefix=stage,
    )
    # Format the result to the format the metric expects.
    if data_args.version_2_with_negative:
        formatted_predictions = [
            {"id": str(k), "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
        ]
    else:
        formatted_predictions = [{"id": str(k), "prediction_text": v} for k, v in predictions.items()]

    references = [{"id": str(ex["id"]), "answers": ex[answer_column_name]} for ex in examples]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references)

In [25]:
data = data.map(prepare_train_features, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## Setting up and Running our Trainer
Now that we have a prepped dataset, let's fine-tune our model!

## Configurando e executando nosso treinador
Agora que temos um conjunto de dados preparado, vamos ajustar nosso modelo!

In [33]:
# Set up our trainer
trainer = QuestionAnsweringTrainer(
    model=model,
    train_dataset=data,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    post_process_function=post_processing_function)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [57]:
# Run the trainer
trainer.train()

# Save our model
trainer.save_model("./ft-distilbert")

Step,Training Loss


In [60]:
# Let's evaluate our model!
# Specify an input question and context
question = "What can an attacker do with XSS?" #O que um invasor pode fazer com XSS?
with open("./data/xss.txt", "r") as f:
    context = f.read()

# Use HuggingFace pipeline to answer the question
question_answerer = pipeline("question-answering", model="./ft-distilbert")
question_answerer(question=question, context=context)
#correct answer: a vítima envia solicitações ao servidor

{'score': 0.003248984459787607,
 'start': 121,
 'end': 185,
 'answer': 'There are three main kinds of XSS:  Once the malicious script is'}

'o invasor pode realizar uma variedade de atividades maliciosas. O atacante

In [52]:
# Let's evaluate our model!
# Specify an input question and context
question = "God exists?"
with open("./data/xss.txt", "r") as f:
    context = f.read()

# Use HuggingFace pipeline to answer the question
question_answerer = pipeline("question-answering", model="./ft-distilbert")
question_answerer(question=question, context=context)

{'score': 0.0022751668002456427,
 'start': 1487,
 'end': 1537,
 'answer': 'The goal is to prevent one site from being able to'}

Hopefully your answer was satisfactory! If not, don't worry about it too much, our dataset was extremely small and we only trained for 3 epochs, so some issues can be expected. This is why so many LLM datasets are so big!

Esperamos que sua resposta tenha sido satisfatória! Caso contrário, não se preocupe muito, nosso conjunto de dados era extremamente pequeno e treinamos apenas por 3 épocas, portanto, alguns problemas podem ser esperados. É por isso que tantos conjuntos de dados LLM são tão grandes!